Code to create a matrix version of np.diff, with automatic zero padding.

In [1]:
import numpy as np
ra = lambda *args: np.random.rand(*args)

x = ra(10)

In [11]:
from scipy.linalg import toeplitz

def diffmat(d, n=1, pad=False, npcompat=True):
    # creates a diff matrix for n fold diff, padding with zeros
    #
    # d : size of axis to differentiate
    # n: number of times to differentiate
    # pad: pad with zeros
    # npcompat: if True, pads with zeros once (numpy behaviour when pre/append=0.0). 
    #    If false, pads every time the derivative is taken.
    d_impulse = np.diff(np.array([*[0]*n, 1.0, *[0]*n ]), n=n)
    col = np.zeros((d+n,))
    col[:n+1]=d_impulse
    row = np.zeros((d,))
    row[0] = col[0]
    dmat = toeplitz(col, row)
    if not pad:
        dmat = dmat[n:d]
    elif npcompat:
        # we have to remove some rows of dmat
        dmat = dmat[n-1:dmat.shape[0]-(n-1)]
    return dmat

shape = lambda x:getattr(x, 'shape', ())
dims = lambda x:getattr(x, 'ndim', 0)

def diff(x, n=1, axis=-1, pad=False, npcompat=True):
    # a variant of np.diff using a matrix, with more padding
    # but only with zeros
    nd = dims(x)
    dmat = diffmat(shape(x)[axis], n=n, pad=pad, npcompat=npcompat)
    didx = 'ijklmnopqrstuvwxyz'[:2]
    idx = 'ijklmnopqrstuvwxyz'[2:nd+2]
    idx = idx.replace(idx[axis],didx[1])
    result = idx.replace(idx[axis],didx[0])
    return np.einsum(didx+','+idx+'->'+result, dmat, x)

In [12]:
# n=1
x = ra(10)
a = np.diff(x)
b = diff(x, pad=False)
print(np.all(np.isclose(a,b)))
a = np.diff(x, prepend=0.0, append=0.0)
b = diff(x, pad=True)
print(np.all(np.isclose(a,b)))

True
True


In [13]:
# n=2
a = np.diff(x, n=2)
b = diff(x, n=2, pad=False)
print(np.all(np.isclose(a,b)))
a = np.diff(x, n=2, prepend=0.0, append=0.0)
b = diff(x, n=2, pad=True)
print(np.all(np.isclose(a,b)))

True
True


In [14]:
# n=3
a = np.diff(x, n=3)
b = diff(x, n=3, pad=False)
print(np.all(np.isclose(a,b)))
a = np.diff(x, n=3, prepend=0.0, append=0.0)
b = diff(x, n=3, pad=True)
print(np.all(np.isclose(a,b)))

True
True


In [15]:
x = ra(10,6)
a = np.diff(x, n=1, axis=1)
b = diff(x, n=1, axis=1, pad=False)
print(np.all(np.isclose(a,b)))
a = np.diff(x, n=1, axis=1, prepend=0.0, append=0.0)
b = diff(x, n=1, axis=1, pad=True)
print(np.all(np.isclose(a,b)))

True
True
